# Major Model Analysis Ahead

In [3]:
from utils.model_analysis import * 
import os
import matplotlib.pyplot as plt
from nilearn import datasets, plotting

### Helper Functions

In [ ]:
def process_subject_data(data,subject, split=True):
    # Define the split dictionary
    split_dict = {"subj01": 19004, "subj02": 19004, "subj03": 19004, "subj04": 19004,
                  "subj05": 19004, "subj06": 18978, "subj07": 19004, "subj08": 18981}


    if subject not in split_dict.keys():
        print("Invalid subject")
        return None, None

    # Split data based on the split dictionary
    if split:
        lh_data = data[:split_dict[subject]]
        rh_data = data[split_dict[subject]:]
    else:
        lh_data = data
        rh_data = None

    # Read ROI directories
    roi_dir_lh = np.load(fr"C:\Users\rvacher\Downloads\algonauts_2023_tutorial_data\{subject}\roi_masks\lh.all-vertices_fsaverage_space.npy")
    if rh_data is not None:
        roi_dir_rh = np.load(fr"C:\Users\rvacher\Downloads\algonauts_2023_tutorial_data\{subject}\roi_masks\rh.all-vertices_fsaverage_space.npy")
    else:
        roi_dir_rh = None

    # Create responses
    fsaverage_response_lh = np.zeros(len(roi_dir_lh))
    fsaverage_response_lh[np.where(roi_dir_lh)[0]] = lh_data

    if rh_data is not None:
        fsaverage_response_rh = np.zeros(len(roi_dir_rh))
        fsaverage_response_rh[np.where(roi_dir_rh)[0]] = rh_data
    else:
        fsaverage_response_rh = None

    return fsaverage_response_lh, fsaverage_response_rh

In [12]:
if __name__ == "__main__":
    #Change as needed
    checkpoint_path = '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_200_EPOCHS100_BATCHSIZE_16_TIME_2023-12-07_23:18:19.pt'
    output_size = 100
    backbone = 'alexnet'
    #load images and original activations
    subject_id = 1
    brain, image_paths = load_subject_data(1, index_start=0, index_end=5)

    #pick image and activation
    image = np.load(image_paths[0])
    image = preprocess(image)

    activation = torch.Tensor(brain[0,:])
    print('activation type:', type(activation))

    scores = {}

    # Check if GPU is available and if not, use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Load in model & Create feature extractor
    trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
    feature_extractor = torch.hub.load('utils', backbone, source = 'local')

    trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
    trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
    trained_model_backbone = trained_model.feature_extractor

    if backbone == "alexnet":
        output_layer_names = get_module_names(trained_model_backbone)[1:-1]
    else:
        output_layer_names = get_block_names(trained_model_backbone)[:-1]

    feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

    #Extract features & predict fMRI data 
    outputs = feature_extractor(image.unsqueeze(0))
    flat_outputs = flatten_features(outputs)
    predictions = make_prediction(trained_model, flat_outputs, 256, subject = subject_id)
    print(predictions)

    #Inverse transform of preds with frozen PCA models
    pca = get_pca_model(subject_id)
    inversed_predictions = predictions.copy()
    for key in inversed_predictions:
        #convert prediction tensors to np arrays to make it compatible for inverse pca
        preds = inversed_predictions[key].detach().numpy()
        preds = torch.Tensor(pca.inverse_transform(preds))
        print('preds type:', type(preds))
        # inverse-pca and store in new dict
        inversed_predictions[key] = preds

   #Caculating MNNPC on Preds
    # for key in inversed_predictions:
    #     preds = inversed_predictions[key]
    #     mnnpc = MNNPC()
    #     score = mnnpc(pred = preds, gt=activation)
    #     scores[key] = score

    # print(scores)

Shape of pca_brain:  (5, 39548)
activation type: <class 'torch.Tensor'>
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x256 and 9216x256)